*Accompanying code examples of the book "Introduction to Artificial Neural Networks and Deep Learning: A Practical Guide with Applications in Python" by [Sebastian Raschka](https://sebastianraschka.com). All code examples are released under the [MIT license](https://github.com/rasbt/deep-learning-book/blob/master/LICENSE). If you find this content useful, please consider supporting the work by buying a [copy of the book](https://leanpub.com/ann-and-deeplearning).*
  
Other code examples and content are available on [GitHub](https://github.com/rasbt/deep-learning-book). The PDF and ebook versions of the book are available through [Leanpub](https://leanpub.com/ann-and-deeplearning).

In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p tensorflow

Sebastian Raschka 

CPython 3.6.0
IPython 6.0.0

numpy 1.12.1
tensorflow 1.1.0


# Model Zoo -- Multilayer Perceptron

### Low-level Implementation

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


##########################
### DATASET
##########################

mnist = input_data.read_data_sets("./", one_hot=True)


##########################
### SETTINGS
##########################

# Hyperparameters
learning_rate = 0.1
training_epochs = 10
batch_size = 128

# Architecture
n_hidden_1 = 128
n_hidden_2 = 256
n_input = 784
n_classes = 10


##########################
### GRAPH DEFINITION
##########################

g = tf.Graph()
with g.as_default():

    # Input data
    tf_x = tf.placeholder(tf.float32, [None, n_input])
    tf_y = tf.placeholder(tf.float32, [None, n_classes])

    # Model parameters
    weights = {
        'h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1], stddev=0.1)),
        'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], stddev=0.1)),
        'out': tf.Variable(tf.truncated_normal([n_hidden_2, n_classes], stddev=0.1))
    }
    biases = {
        'b1': tf.Variable(tf.zeros([n_hidden_1])),
        'b2': tf.Variable(tf.zeros([n_hidden_2])),
        'out': tf.Variable(tf.zeros([n_classes]))
    }

    # Multilayer perceptron
    layer_1 = tf.add(tf.matmul(tf_x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']

    # Loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out_layer, labels=tf_y))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

    # Prediction
    correct_prediction = tf.equal(tf.argmax(tf_y, 1), tf.argmax(out_layer, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    
##########################
### TRAINING & EVALUATION
##########################

with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = mnist.train.num_examples // batch_size

        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, c = sess.run([optimizer, cost], feed_dict={tf_x: batch_x,
                                                          tf_y: batch_y})
            avg_cost += c
        
        train_acc = sess.run(accuracy, feed_dict={tf_x: mnist.train.images,
                                                    tf_y: mnist.train.labels})
        valid_acc = sess.run(accuracy, feed_dict={tf_x: mnist.validation.images,
                                                    tf_y: mnist.validation.labels})  
        
        print("Epoch: %03d | AvgCost: %.3f" % (epoch + 1, avg_cost / (i + 1)), end="")
        print(" | Train/Valid ACC: %.3f/%.3f" % (train_acc, valid_acc))
        
    test_acc = sess.run(accuracy, feed_dict={tf_x: mnist.test.images,
                                             tf_y: mnist.test.labels})
    print('Test ACC: %.3f' % test_acc)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz
Epoch: 001 | AvgCost: 0.443 | Train/Valid ACC: 0.926/0.928
Epoch: 002 | AvgCost: 0.218 | Train/Valid ACC: 0.948/0.953
Epoch: 003 | AvgCost: 0.167 | Train/Valid ACC: 0.961/0.961
Epoch: 004 | AvgCost: 0.137 | Train/Valid ACC: 0.966/0.964
Epoch: 005 | AvgCost: 0.115 | Train/Valid ACC: 0.972/0.967
Epoch: 006 | AvgCost: 0.099 | Train/Valid ACC: 0.975/0.970
Epoch: 007 | AvgCost: 0.086 | Train/Valid ACC: 0.978/0.972
Epoch: 008 | AvgCost: 0.077 | Train/Valid ACC: 0.981/0.974
Epoch: 009 | AvgCost: 0.068 | Train/Valid ACC: 0.982/0.974
Epoch: 010 | AvgCost: 0.061 | Train/Valid ACC: 0.985/0

### tensorflow.layers Abstraction

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


##########################
### DATASET
##########################

mnist = input_data.read_data_sets("./", one_hot=True)


##########################
### SETTINGS
##########################

# Hyperparameters
learning_rate = 0.1
training_epochs = 10
batch_size = 128

# Architecture
n_hidden_1 = 128
n_hidden_2 = 256
n_input = 784
n_classes = 10


##########################
### GRAPH DEFINITION
##########################

g = tf.Graph()
with g.as_default():

    # Input data
    tf_x = tf.placeholder(tf.float32, [None, n_input])
    tf_y = tf.placeholder(tf.float32, [None, n_classes])

    # Multilayer perceptron
    layer_1 = tf.layers.dense(tf_x, n_hidden_1, activation=tf.nn.relu, name='layer_1')
    layer_2 = tf.layers.dense(layer_1, n_hidden_2, activation=tf.nn.relu)
    out_layer = tf.layers.dense(layer_2, n_classes, activation=None)

    # Loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out_layer, labels=tf_y))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

    # Prediction
    correct_prediction = tf.equal(tf.argmax(tf_y, 1), tf.argmax(out_layer, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


##########################
### TRAINING & EVALUATION
##########################
    
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = mnist.train.num_examples // batch_size

        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, c = sess.run([optimizer, cost], feed_dict={tf_x: batch_x,
                                                          tf_y: batch_y})
            avg_cost += c
        
        train_acc = sess.run(accuracy, feed_dict={tf_x: mnist.train.images,
                                                    tf_y: mnist.train.labels})
        valid_acc = sess.run(accuracy, feed_dict={tf_x: mnist.validation.images,
                                                    tf_y: mnist.validation.labels})  
        
        print("Epoch: %03d | AvgCost: %.3f" % (epoch + 1, avg_cost / (i + 1)), end="")
        print(" | Train/Valid ACC: %.3f/%.3f" % (train_acc, valid_acc))
        
    test_acc = sess.run(accuracy, feed_dict={tf_x: mnist.test.images,
                                             tf_y: mnist.test.labels})
    print('Test ACC: %.3f' % test_acc)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Epoch: 001 | AvgCost: 0.479 | Train/Valid ACC: 0.922/0.928
Epoch: 002 | AvgCost: 0.224 | Train/Valid ACC: 0.940/0.944
Epoch: 003 | AvgCost: 0.170 | Train/Valid ACC: 0.956/0.958
Epoch: 004 | AvgCost: 0.137 | Train/Valid ACC: 0.967/0.966
Epoch: 005 | AvgCost: 0.114 | Train/Valid ACC: 0.971/0.969
Epoch: 006 | AvgCost: 0.097 | Train/Valid ACC: 0.977/0.973
Epoch: 007 | AvgCost: 0.085 | Train/Valid ACC: 0.979/0.976
Epoch: 008 | AvgCost: 0.075 | Train/Valid ACC: 0.983/0.977
Epoch: 009 | AvgCost: 0.066 | Train/Valid ACC: 0.985/0.978
Epoch: 010 | AvgCost: 0.059 | Train/Valid ACC: 0.987/0.978
Test ACC: 0.975
